In [1]:
from selenium import webdriver
from time import sleep

In [2]:
USERNAME = "xxxxxxxx"
PASSWORD = "xxxxxxxx"

In [3]:
driver = webdriver.Chrome(r"C:\Users\user\chromedriver-win64\chromedriver-win64\chromedriver.exe")
target_url = "https://twitter.com/login"
driver.get(target_url)
sleep(3)

In [4]:
error_flg = False

##### ---自動ログイン処理

In [5]:
try:

    username_input = driver.find_element_by_xpath("//input[@autocomplete='username']")
    username_input.send_keys(USERNAME)
    sleep(2)
    next_button = driver.find_element_by_xpath("//span[text()='次へ']")
    next_button.click()
    sleep(3)

    password_input = driver.find_element_by_xpath("//input[@autocomplete='current-password']")
    password_input.send_keys(PASSWORD)
    sleep(2)
    login_button = driver.find_element_by_xpath("//span[text()='ログイン']")
    login_button.click()
    sleep(3)

except Exception as e:
    
    error_flg = True
    print(f"エラーが発生しました", e)

In [6]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

##### ---削除処理関数

In [7]:
def delete_post(driver, post):
    try:
        more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//div[@aria-label='もっと見る']")))
        driver.execute_script("arguments[0].click();", more_button)
        sleep(1)

        for _ in range(2):
            delete_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//span[text()='削除']")))
            driver.execute_script("arguments[0].click();", delete_button)
            sleep(1)
    except Exception as e:
        print(f"削除処理でエラーが発生しました:", e)

##### ---リポスト取り消し処理関数

In [8]:
def undo_repost(driver, post):
    try:
        repost_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//div[@data-testid='unretweet']")))
        driver.execute_script("arguments[0].click();", repost_button)
        sleep(1)

        undo_repost = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//span[text()='ポストを取り消す']")))
        driver.execute_script("arguments[0].click();", undo_repost)
        sleep(1)
    except Exception as e:
        print(f"リポスト取り消し処理でエラーが発生しました:", e)

##### ---プロフィールページに移動して実行

In [10]:
if not error_flg:
    try:
        profile_url = "https://twitter.com/xxxxxxxx"
        driver.get(profile_url)
        sleep(3)

        i = 0
        while True:
            posts = driver.find_elements_by_xpath(".//div[@data-testid='cellInnerDiv']")
            if i >= len(posts):
                break

            post = posts[i]
            try:
                sleep(1)
                you_reposted = post.find_element_by_xpath("//span[text()='あなたがリポストしました']")
            except NoSuchElementException:
                you_reposted = None

            if not you_reposted:
                delete_post(driver, post)
            else:
                undo_repost(driver, post)

            i += 1

            if i > 100:
                break

    except Exception as e:
        error_flg = True
        print(f"エラーが発生しました:", e)